In [1]:
import sys
import os
sys.path.insert(0, os.path.abspath("../../"))


In [2]:
from brmspy.brms import bf
from brmspy import brms

In [3]:
with brms.manage(environment_name="mrp") as ctx:
    ctx.install_brms(use_prebuilt=True)
    ctx.install_rpackage("MCMCglmm")
    

REMOTE CALL


[brmspy][install_brms] Activating runtime /Users/sebastian/.brmspy/runtime/macos-arm64-r4.5-0.2.0
[R]: Error in loadNamespace(x) : there is no package called ‘cmdstanr’
  
[brmspy][install_brms] lib paths are ['/Users/sebastian/.brmspy/environment/mrp/Rlib', '/Library/Frameworks/R.framework/Versions/4.5-arm64/Resources/library', '/Users/sebastian/.brmspy/runtime/macos-arm64-r4.5-0.2.0/Rlib']
[brmspy][install_brms] Setting cmdstan path to /Users/sebastian/.brmspy/runtime/macos-arm64-r4.5-0.2.0/cmdstan


REMOTE CALL
REMOTE CALL
lib path is ['/Users/sebastian/.brmspy/environment/mrp/Rlib']


[brmspy][worker_main] MCMCglmm 2.36 already installed.


In [4]:
status = brms.status()

REMOTE CALL


In [6]:
status

RuntimeStatus(active_runtime=PosixPath('/Users/sebastian/.brmspy/runtime/macos-arm64-r4.5-0.2.0'), is_activated=False, system=SystemInfo(os='macos', arch='arm64', r_version=(4, 5, 2), fingerprint='macos-arm64-r4.5', glibc_version=None, clang_version=(17, 0), gxx_version=None, has_rtools=False), can_use_prebuilt=True, prebuilt_available=True, compatibility_issues=(), installed_runtimes=(PosixPath('/Users/sebastian/.brmspy/runtime/test-x86_64-r4.3-0.1.0'), PosixPath('/Users/sebastian/.brmspy/runtime/macos-arm64-r4.5-0.2.0')), brms_version='2.23.0', cmdstanr_version=None, rstan_version='2.32.7')

In [5]:
data = brms.get_brms_data("epilepsy")

REMOTE CALL


In [ ]:
r1 = brms.brm(
    formula="count ~ zAge + zBase * Trt + (1|patient)",
    data=data,
    family="poisson",
    warmup=500,
    iter=1000,
    chains=4
)

r2 = brms.brm(
    formula="count ~ zAge + zBase * Trt",
    data=data,
    family="poisson",
    warmup=500,
    iter=1000,
    chains=4
)

REMOTE CALL


[brmspy][worker_main] Fitting model with brms (backend: cmdstanr)...
[R]: Compiling Stan program...
  


In [ ]:
brms.summary(r1)

cmd CALL


 Family: poisson 
  Links: mu = log 
Formula: count ~ zAge + zBase * Trt + (1 | patient) 
   Data: structure(list(Age = c(31, 30, 25, 36, 22, 29, 31, (Number of observations: 236) 
  Draws: 4 chains, each with iter = 1000; warmup = 500; thin = 1;
         total post-warmup draws = 2000

Multilevel Hyperparameters:
~patient (Number of levels: 59) 
              Estimate Est.Error l-95% CI u-95% CI Rhat Bulk_ESS Tail_ESS
sd(Intercept)     0.59      0.07     0.46     0.73 1.00      529      835

Regression Coefficients:
          Estimate Est.Error l-95% CI u-95% CI Rhat Bulk_ESS Tail_ESS
Intercept     2.05      0.27     1.47     2.56 1.01      392      597
zAge          0.10      0.09    -0.07     0.29 1.01      395      810
zBase         0.63      0.26     0.11     1.16 1.02      347      579
Trt          -0.27      0.17    -0.61     0.07 1.01      425      718
zBase:Trt     0.06      0.16    -0.27     0.37 1.02      374      582

Draws were sampled using sample(hmc). For each parameter

In [ ]:
loo_result = brms.call("loo", r1, r2)
df = loo_result['ic_diffs__']
df.index.values[0] = str(df.index.values[0])[:32]
df

cmd CALL


R callback write-console: In addition:   
R callback write-console: There were 14 warnings (use warnings() to see them)  
R callback write-console: 
  


,LOOIC,SE
structure(list(formula = structu,-411.848552,129.229581
